# Objective
What is the objective or goal that you are trying to accomplish? What is the decision that you must make?

In [ ]:
#The goal I am trying to achieve is to reduce the number of calls and resolve them at the first point of contact so that the customers are satisfied

# Hypothesis: Research Question?
 What is the question that you would like to answer in order to make a decision.

In [ ]:
#What is the reason for so many calls?

#In the internet area with the most difficulties

# Data Source
Explain where did you get the data. How can you trust this data? Who produced this data and what were their motiviations?


In [ ]:
#The data I got from the customer service of Digicel 
#I trust the data because it comes from the call center and it's specific customer requests
#The data was produced by the agents in order to note the requests of the customers and to solve their difficulties

# Data Cleaning
In this step you will prepare your data for analysis.

## Review data types
Inspect the dataset for the data types of each column.

## Analytical Transformations
Perform any transformation on the columns in the dataset to enable further analysis.

### Treatment of Missing Values
If there are any missing values, how do you plan to treat those data columns?

In [ ]:
!pip install geopandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import datetime as dt
import pickle

In [ ]:
dec = pd.read_excel("decembre.xlsx")
nov = pd.read_excel("nov 11.xlsx")
nov_2=pd.read_excel("novembre.xlsx")
octo = pd.read_excel("octobre.xlsx")
jan  =pd.read_excel("janvier.xlsx")
fev  =pd.read_excel("fevrier.xlsx")
Mar  =pd.read_excel("mars.xlsx")

In [ ]:
fev.columns=dec.columns
Mar.columns=dec.columns

In [ ]:
print("sub category null for dec",sum(dec.CallSubCategory.isna()))

In [ ]:
print("sub category null for fev",sum(fev.CallSubCategory.isna()))

In [ ]:
nov.CallCategory.unique()

In [ ]:
fev

In [ ]:
#We will aggregate the data for the last 6 months of the year 2020, 2021
unified=pd.concat([octo, nov,nov_2, dec,jan,fev,Mar], axis=0, ignore_index= True)

In [ ]:
unified.head(2)

In [ ]:
unified['DateAndTime'] = unified.CreatedDate.str.cat(unified.CreatedTime,sep=" ")

In [ ]:
unified.columns

In [ ]:
unified.info()

In [ ]:
unified.DateAndTime=pd.to_datetime(unified.DateAndTime)

In [ ]:
unified["month"]=unified.DateAndTime.dt.month
unified["day"]=unified.DateAndTime.dt.day
unified["hour"]=unified.DateAndTime.dt.hour
unified["year"]=unified.DateAndTime.dt.year

In [ ]:
#unified.set_inde([])

In [ ]:
unified.DateAndTime

In [ ]:
CallCategory=unified[['DateAndTime','CallCategory' ]].set_index('DateAndTime')

In [ ]:
CallCategory.resample("D").count().plot(figsize=(15,15))

In [ ]:
#Format Date

unified['CreatedDate']=pd.to_datetime(unified["CreatedDate"])


In [ ]:
#Let's calculate the number of calls per day
dailycalls = unified.CreatedDate.value_counts()
dailycalls = dailycalls.to_frame().reset_index()
dailycalls.sort_values("index", inplace=True)
dailycalls

In [ ]:
dailycalls.set_index('index')
sns.lineplot(data = dailycalls,x="index",y="CreatedDate")

In [ ]:
#Let's note the days with the most calls
dailycalls2 = unified.CreatedDate.value_counts()
dailycalls2 = dailycalls2.to_frame().reset_index()
dailycalls2.sort_values("CreatedDate", ascending = False, inplace=True)
dailycalls2

In [ ]:
sns.barplot(data=dailycalls2.head(5),y="index",x="CreatedDate")

In [ ]:
df_2021_02_01 = unified.loc[unified['CreatedDate'] == '2021-02-01']


In [ ]:
test=unified.set_index(['DateAndTime'])#.resample("D").count()

In [ ]:
#Let's check for each service the number of requests received per day
daySub=pd.pivot_table(unified, index='DateAndTime', values='CallCategory',columns="CallSubCategory", aggfunc='count').resample("D").count()

In [ ]:
daySub=pd.pivot_table(unified, index='DateAndTime', values='CallCategory',columns="CallSubCategory", aggfunc='count').resample("D").count()

In [ ]:
daySub

In [ ]:
daySub["Total"]=daySub.sum(axis=1)

In [ ]:
daySub.sort_values(by="Total")

In [ ]:
#Among the 5 days with the most calls let's check the cause
#Day1=2021-02-01
#Day2=2021-02-13
#Day3=2021-03-24
#Day4=2021-03-26
#Day5=2021-03-25	


In [ ]:
daySub.loc["2021-02-01",:]#.sort_index()

In [ ]:
daySub.loc["2021-02-13",:]#.sort_index()

In [ ]:
daySub.loc["2021-03-24",:]#.sort_index()

In [ ]:
daySub.loc["2021-03-26",:]#.sort_index()

In [ ]:
daySub.loc["2021-03-25",:]#.sort_index()

In [ ]:
#Amount of calls per day
daySub.sum(axis=1)

In [ ]:
#The list of the 5 days with the most calls
df = dailycalls2.iloc[:5, :]
df.rename(columns={"index":"Jour"}, inplace=True)
df.Jour = df.Jour.astype("object")
df.Jour = df.Jour.apply(lambda x: str(x)[:10])
b = df.Jour.to_list()
Jour1 = b[0]
Jour2 = b[1]
Jour3 = b[2:]
df1 = unified[unified["CreatedDate"]== Jour1]
df2 = unified[unified["CreatedDate"]== Jour2]
df3 = pd.DataFrame()
for Jour in Jour3:
    df0 = unified[unified["CreatedDate"]== Jour]
    df3 = pd.concat([df3, df0])

In [ ]:
test=unified.set_index("DateAndTime")

In [ ]:
#Let's see which time slot has the most calls
test.CallCategory.resample("H").count().sort_values(ascending=False)

In [ ]:
#let's see in the category complaint the most frequent requests of the customers
unified1 = unified[unified["Callsubcategory2"]=="Complaint"]

In [ ]:
Callsubcategory1 =unified1.groupby(by=['Callsubcategory1']).size().to_frame()
Callsubcategory1.columns = ['Total']
Callsubcategory1.sort_values(by=['Total'],inplace=False)
Callsubcategory1

In [ ]:
unified5= unified1.groupby(["Callsubcategory1","Callsubcategory2"])["CreatedDate"].count().to_frame()
unified5

In [ ]:
#let's see in the category request the most frequent requests of the customers
unified3 = unified[unified["Callsubcategory2"]=="Request"]

In [ ]:
Callsubcategory1 =unified3.groupby(by=['Callsubcategory1']).size().to_frame()
Callsubcategory1.columns = ['Total']
Callsubcategory1.sort_values(by=['Total'],inplace=False)
Callsubcategory1

In [ ]:
unified5= unified3.groupby(["Callsubcategory1","Callsubcategory2"])["CreatedDate"].count().to_frame()
unified5

In [ ]:
##let's see in the query the most frequent requests of the customers
unified4 = unified[unified["Callsubcategory2"]=="Query"]

In [ ]:
Callsubcategory1 =unified4.groupby(by=['Callsubcategory1']).size().to_frame()
Callsubcategory1.columns = ['Total']
Callsubcategory1.sort_values(by=['Total'],inplace=False)
Callsubcategory1

In [ ]:
unified5= unified4.groupby(["Callsubcategory1","Callsubcategory2"])["CreatedDate"].count().to_frame()
unified5

In [ ]:
display(unified1)

In [ ]:
unified.CreatedDate.dtypes

In [ ]:
unified.info()

In [ ]:
def generate_barchart(data=unified, title ="",abs_value ="Total",rel_value="Percent",figsize =(10,6)):
     plt.figure(figsize=figsize)
     axes = sns.barplot(data=data,x=data.index,y=abs_value)
     i=0
     for tot, perc in zip(data[abs_value],data[rel_value]):
        axes.text(i,
           tot/2,
              str(np.round(perc*100,2))+ "%",
               fontdict=dict(color='yellow',fontsize=12,horizontalalignment="center")
              )
        axes.text(i,
           tot+10,
              str(tot),
               fontdict=dict(color='blue',fontsize=12,horizontalalignment="center")
              )
        i+=1
     plt.title(title)
     plt.show()

In [ ]:
#let's see when customers call for which service they require the most
unified['CallCategory'] =unified['CallCategory'].apply(lambda x: str(x).strip().replace("\\t",""))

CallCategory =unified.groupby(by=['CallCategory']).size().to_frame()
CallCategory.columns = ['Total']
CallCategory.sort_values(by=['Total'],inplace=False)
CallCategory

In [ ]:
CallCategory =CallCategory.tail(10)
sns.barplot(x=CallCategory.index,y=CallCategory["Total"])

In [ ]:
month=pd.pivot_table(unified, index="month", columns="CallCategory", values="CaseNumber", aggfunc="count")

In [ ]:
sns.lineplot(data=month)

In [ ]:
#Let's see which of the Dealer/Vendor and Prepaid/Postpaid customers uses the service the most
HelpTopic =unified.groupby(by=['HelpTopic']).size().to_frame()
HelpTopic.columns = ['Total']
HelpTopic.sort_values(by=['Total'],inplace=False)
HelpTopic

In [ ]:
HelpTopic =HelpTopic.iloc[::-1,:]
sns.barplot(data=HelpTopic,y=HelpTopic.index,x="Total")

In [ ]:
#Let's look for the most frequent customer requests
Callsubcategory2 =unified.groupby(by=['Callsubcategory2']).size().to_frame()
Callsubcategory2.columns = ['Total']
Callsubcategory2.sort_values(by=['Total'],inplace=False)
Callsubcategory2

In [ ]:
Callsubcategory2 =Callsubcategory2.iloc[::-1,:]
sns.barplot(data=Callsubcategory2,y=Callsubcategory2.index,x="Total")

In [ ]:
#What do customers complain about when they call the service
CallSubCategory =unified.groupby(by=['CallSubCategory']).size().to_frame()
CallSubCategory.columns = ['Total']
CallSubCategory.sort_values(by=['Total'],inplace=True)
CallSubCategory

In [ ]:
CallSubCategory =CallSubCategory.tail(10)
sns.barplot(data=CallSubCategory,y=CallSubCategory.index,x="Total")

In [ ]:
#Let's create a dictionary to better explain the reasons for the calls.
dict ={ "Calls":"when the customer calls you and the call is interrupted", "Depletion":"Unable to make calls to Digicel numbers","Digi_kredi":"to be done in Digicel service","Rederect":"information needed to transport to another department","Barring/Unbarring":"unblock and block an account"}

In [ ]:
#Let's see the customers by category and which one calls the most
SkillSet =unified.groupby(by=['SkillSet']).size().to_frame()
SkillSet.columns = ['Total']
SkillSet.sort_values(by=['Total'],inplace=True)
SkillSet

In [ ]:
SkillSet =SkillSet.tail(10)
sns.barplot(data=SkillSet,y=SkillSet.index,x="Total")

In [ ]:
#Let's create a dictionary to better explain the types of customers
dict= {"Pre_S01_VHV":"2nd customers who call much more for the Diaspora Offer service","PRE_S02_HV":"1nd customers who call much more for the Diaspora Offer service","PRE_SO3_MV":" customers who call much more for the 4G Services","PRE_S04-LV":" 2nd customer who calls much more for the 4 service Lte and the Disputed charges service","PRE_S05_VLV":"Customer5","PRE_DO_Line":"Ccustomers who call much more for the Disputed charges service","DO_Line":"customers who call much more for the Unbarring service","UDP_VENDORLINE":"Vendor","DealerLine":"replacement","Sim_Depletion":"Unable to make calls to Digicel numbers"}

In [ ]:
#ns pral gade pou chak SkillSet yo pou kisa kliyan yo rele plis pou nou kapab idantifye ki kliyan yo ye
pd.crosstab(unified['SkillSet'], unified['CallSubCategory'])

In [ ]:
#let's see where there is much more of a Network problem
CallSubCategory_Location =unified[unified['CallSubCategory'] ==' Network Issue '].groupby(by=['CallSubCategory','Location']).size().to_frame()
CallSubCategory_Location .columns = ['Total']
CallSubCategory_Location .sort_values(by=['Total'],inplace=True)
CallSubCategory_Location  = CallSubCategory_Location.reset_index(level = 1)

In [ ]:
CallSubCategory_Location

In [ ]:
CallSubCategory_Location.tail(6).plot(kind = 'bar', x = 'Location')

In [ ]:
#let's see the phone model used by these people 
CallSubCategory_Handset =unified[unified['CallSubCategory'] ==' Network Issue '].groupby(by=['CallSubCategory','Handset']).size().to_frame()
CallSubCategory_Handset .columns = ['Total']
CallSubCategory_Handset .sort_values(by=['Total'],inplace=True)
CallSubCategory_Handset  = CallSubCategory_Handset.reset_index(level = 1)

In [ ]:
CallSubCategory_Handset

In [ ]:
CallSubCategory_Handset.tail(6).plot(kind = 'bar', x = 'Handset')

In [ ]:
unified.CallSubCategory.unique()

In [ ]:
unified[unified.Status=='Dropped Call']

In [ ]:
unified.SkillSet.unique()

In [ ]:
unified.AgentComment

In [ ]:
unified.columns

In [ ]:
#I will choose the columns I will work with
columns=[]
new_unified=unified[["CreatedDate","CreatedTime","CallCategory","HelpTopic","CallSubCategory","Callsubcategory1","Callsubcategory2","SkillSet","Location","Handset","CallingNumber","AgentComment","AgentFullname","Status"]]
new_unified

In [ ]:
#I made transformations on the columns with missing values
new_unified[['Location','Handset','Callsubcategory1']]= new_unified[['Location','Handset','Callsubcategory1']].fillna('None')

In [ ]:
new_unified


In [ ]:
unified["CreatedDate"].value_counts()

In [ ]:
unified["Month"]=unified["CreatedDate"].apply(lambda x: str(x)[5:7])
unified["Month"].value_counts()

In [ ]:
table = pd.pivot_table(unified, values='CallingNumber', index=['Month'],columns=['CallCategory'], aggfunc= "count")
table

In [ ]:
unified["CreatedDate"]=pd.to_datetime(unified["CreatedDate"])
unified["Month"] = unified["CreatedDate"].dt.month
unified["Year"] = unified["CreatedDate"].dt.year


In [ ]:
by_month = unified.groupby(by=["Month","Year"])['CallCategory'].value_counts().to_frame()
by_month#.xs("Network", level=2)

In [ ]:
#
by_month.plot(kind = 'bar', figsize=(15,15))

In [ ]:
#Turn customer numbers into ID
def myHash(x):

    return hash(str(x))
new_unified['CallingNumber']=new_unified['CallingNumber'].apply(myHash)
new_unified

In [ ]:
#Transform AgentFullname names into IDs
def myHash(x):

    return hash(str(x))
new_unified['AgentFullname']=new_unified['AgentFullname'].apply(myHash)
new_unified

In [ ]:
new_unified["AgentComment"]=new_unified["AgentComment"].astype('str')

In [ ]:
#In the AgentComment column there were customer numbers I had to remove them
def remov(x) :
    result = ''.join([i for i in x if not i.isdigit()])
    return result

In [ ]:
new_unified["AgentComment"]=new_unified["AgentComment"].apply(lambda x : remov(x))

In [ ]:
new_unified

In [ ]:
#new_unified=new_unified[columns]

In [ ]:
#Let's display the AgentComment column
new_unified['AgentComment']

In [ ]:
import seaborn as sns
sns.countplot(data = new_unified, x='Callsubcategory2')

In [ ]:
#Let's create several tabs in order to link to the main Power BI tab
#CallCategory
df_for_PBI = new_unified.iloc[:,:]

cat = df_for_PBI.CallCategory.unique()
cat

In [ ]:
callCatDf = pd.DataFrame({"CallCategory": cat})
callCatDf

In [ ]:
callCatPBI = callCatDf.reset_index()

In [ ]:
callCatDf.reset_index(inplace=True)
callCatDf

In [ ]:
callCatDf.set_index('CallCategory', inplace=True)
callCatDf

In [ ]:
callCatDict = callCatDf.to_dict()
callCatDict

In [ ]:
callCatDict['index']

In [ ]:
df_for_PBI["CallCategory"] = df_for_PBI.CallCategory.replace(callCatDict['index'])

In [ ]:
df_for_PBI

In [ ]:
########CallSubCategory
df_for_PBI = new_unified.iloc[:,:]
callSub = df_for_PBI.CallSubCategory.unique()
callSub 
callSubDf = pd.DataFrame({"CallSubCategory": callSub})
callSubDf
callSubPBI = callSubDf.reset_index()
callSubDf.reset_index(inplace=True)
callSubDf
callSubDf.set_index('CallSubCategory', inplace=True)
callSubDf
callSubDict = callSubDf.to_dict()
callSubDict
callSubDict['index']
df_for_PBI["CallSubCategory"] = df_for_PBI.CallSubCategory.replace(callSubDict['index'])
df_for_PBI


In [ ]:
###Callsubcategory1
df_for_PBI = new_unified.iloc[:,:]
callsub1 = df_for_PBI.Callsubcategory1.unique()
callsub1
callsub1Df = pd.DataFrame({"Callsubcategory1": callsub1})
callsub1Df
callsub1PBI = callsub1Df.reset_index()
callsub1Df.reset_index(inplace=True)
callsub1Df
callsub1Df.set_index('Callsubcategory1', inplace=True)
callsub1Df
callsub1Dict = callsub1Df.to_dict()
callsub1Dict
callsub1Dict['index']
df_for_PBI["Callsubcategory1"] = df_for_PBI.Callsubcategory1.replace(callsub1Dict['index'])
df_for_PBI



In [ ]:
###Callsubcategory2
df_for_PBI = new_unified.iloc[:,:]
Callsub2 = df_for_PBI.Callsubcategory2.unique()
Callsub2
Callsub2Df = pd.DataFrame({"Callsubcategory2": Callsub2})
Callsub2Df
Callsub2PBI = Callsub2Df.reset_index()
Callsub2Df.reset_index(inplace=True)
Callsub2Df
Callsub2Df.set_index('Callsubcategory2', inplace=True)
Callsub2Df
Callsub2Dict = Callsub2Df.to_dict()
Callsub2Dict
Callsub2Dict['index']
df_for_PBI["Callsubcategory2"] = df_for_PBI.Callsubcategory2.replace(Callsub2Dict['index'])
df_for_PBI


In [ ]:
##Location
df_for_PBI = new_unified.iloc[:,:]
loc = df_for_PBI.Location.unique()
loc
locDf = pd.DataFrame({"Location": loc})
locDf
locPBI = locDf.reset_index()
locDf.reset_index(inplace=True)
locDf
locDf.set_index('Location', inplace=True)
locDf
locDict = locDf.to_dict()
locDict
locDict['index']
df_for_PBI["Location"] = df_for_PBI.Location.replace(locDict['index'])
df_for_PBI



In [ ]:
#Handset
df_for_PBI = new_unified.iloc[:,:]
Handset = df_for_PBI.Handset.unique()
Handset
HandsetDf = pd.DataFrame({"Handset": Handset})
HandsetDf
HandsetPBI = HandsetDf.reset_index()
HandsetDf.reset_index(inplace=True)
HandsetDf
HandsetDf.set_index('Handset', inplace=True)
HandsetDf
HandsetDict = HandsetDf.to_dict()
HandsetDict
HandsetDict['index']
df_for_PBI["Handset"] = df_for_PBI.Handset.replace(HandsetDict['index'])
df_for_PBI


In [ ]:
###CreatedDate
df_for_PBI = new_unified.iloc[:,:]
CreatedDate	 = df_for_PBI.CreatedDate.unique()
CreatedDate	
CreatedDateDf = pd.DataFrame({"CreatedDate": CreatedDate})
CreatedDateDf
CreatedDatetPBI = CreatedDateDf.reset_index()
CreatedDateDf.reset_index(inplace=True)
CreatedDateDf
CreatedDateDf.set_index('CreatedDate', inplace=True)
CreatedDateDf
CreatedDateDict = CreatedDateDf.to_dict()
CreatedDateDict
CreatedDateDict['index']
df_for_PBI["CreatedDate"] = df_for_PBI.CreatedDate.replace(CreatedDateDict['index'])
df_for_PBI

In [ ]:
df_for_PBI.to_csv('df_for_PBI.csv', header=True, index=False)

## Data Analysis
Explore the dataset to discover relationships between records or columns and patterns within the data.

## Descriptive Statistical Analysis
Using basic statistical measures such as measurements of central tendancy such as mean, median and mode.

### Distribution of Variables
Identify the distribution of the data to understand the range of values and how the data is structured.

### Outliers in the dataset
Identify if there are any outliers in the dataset based on statistical measures.

# Reflections
## Summary of Data Analysis
- What insights should the user takeaway from EDA.

## Questions unanswered
- What aspects of the research question were we unable to answer and why?

## Recommendations
- What should the reader do next with this information?

## Next Steps
- What will the analyst do next based on the analysis?

In [ ]:
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("fr_core_news_sm")

# Process whole documents
text = (new_unified['AgentComment'].iloc[0])
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

In [ ]:
new_unified['AgentComment'].iloc[0]

In [ ]:
new_unified.columns 

In [ ]:
#datatoexcel = pd.ExcelWriter("PowerBI_capstone.xlsx")

new_unified.to_excel('PowerBI_capstone.xlsx')
#datatoexcel.save()